<a href="https://colab.research.google.com/github/andreidemianko/-Nu-shtozh-Sirius-AI/blob/main/Nu_chtozsh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U accelerate bitsandbytes git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.2/333.2 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 96.8 MB/s eta 0:00:00


In [2]:
from huggingface_hub import notebook_login

notebook_login()

vlm

In [8]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [9]:
from datasets import load_dataset

ds = load_dataset("ceyda/fashion-products-small")

README.md:   0%|          | 0.00/151 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


dataset_infos.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

(…)-00000-of-00002-357f4cbabe1a8ea6.parquet:   0%|          | 0.00/298M [00:00<?, ?B/s]

(…)-00001-of-00002-cbe936f1880f5e72.parquet:   0%|          | 0.00/299M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/42700 [00:00<?, ? examples/s]

In [21]:
import torch
import numpy as np
from PIL import Image
import requests

promt = "Make a description of each item of clothing in the photo. For each item of clothing, describe what it is, color, style, model. If there is a person in the photo, then it is not necessary to describe him. The response should be a list of strings."
#img_url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/tasks/bee.JPG?download=true"
#input_image = Image.open(requests.get(img_url, stream=True).raw)

from transformers import AutoTokenizer, PaliGemmaForConditionalGeneration, PaliGemmaProcessor
import torch

device_vlm = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_id_vlm = "google/paligemma-3b-mix-224"
model_vlm = PaliGemmaForConditionalGeneration.from_pretrained(model_id_vlm, torch_dtype=torch.bfloat16)
processor_vlm = PaliGemmaProcessor.from_pretrained(model_id_vlm)

def discribe_picture(input_image):
  inputs = processor_vlm(text=promt, images=input_image,
                  padding="longest", do_convert_rgb=True, return_tensors="pt").to("cuda")
  model_vlm.to(device_vlm)
  inputs = inputs.to(dtype=model_vlm.dtype)

  with torch.no_grad():
    output = model_vlm.generate(**inputs, max_length=496)

  return(processor_vlm.decode(output[0], skip_special_tokens=True))

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
!pip install open_clip_torch

clip

In [14]:
from transformers import AutoModel, AutoProcessor
model_clip = AutoModel.from_pretrained('Marqo/marqo-fashionCLIP', trust_remote_code=True)
processor_clip = AutoProcessor.from_pretrained('Marqo/marqo-fashionCLIP', trust_remote_code=True)

import torch
from PIL import Image

#img_url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/tasks/bee.JPG?download=true"
#image = Image.open(requests.get(img_url, stream=True).raw)
#text = ["a hat", "a t-shirt", "shoes"]
def find_pictures_with_elements(text):
  urls = []
  for instance in ds:
    image = Image.open(requests.get(instance['link'], stream=True).raw)
    processed = processor_clip(text=text, images=image, padding='max_length', return_tensors="pt")

    with torch.no_grad():
      image_features = model_clip.get_image_features(processed['pixel_values'], normalize=True)
      text_features = model_clip.get_text_features(processed['input_ids'], normalize=True)

      text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    for probe in text_probs:
      if int(probe) >= 0.9:
        urls.append(instance['link'])
        continue
  if len(urls) > 8:
    return urls[:8]
  else:
    return urls


You are using a model of type clip to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
You are using a model of type clip to instantiate a model of type . This is not supported for all configurations of models and can yield errors.


In [9]:
def search_similar_images(input_image):
  descr = discribe_picture(input_image)
  print(descr)
  #urls = find_pictures_with_elements(descr)
  #print(*urls, sep='\n')
  #return urls

In [23]:
url_test = "https://i.pinimg.com/736x/75/39/0d/75390dab4e7ee5e9d5600d46eb7f3d34.jpg"
image_test = Image.open(requests.get(url_test, stream=True).raw)
search_similar_images(image_test)

You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text and `<bos>` token after that. For this call, we will infer how many images each text has and add special tokens.


Make a description of each item of clothing in the photo. For each item of clothing, describe what it is, color, style, model. If there is a person in the photo, then it is not necessary to describe him. The response should be a list of strings.
suit
